In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from model.dmd import ROOT, extract_frames


In [13]:
session = 'gA_1_s2_2019-03-08T09;21;03+01;00'
input_video_path = ROOT / session / f'{session}_rgb_body.mp4'
annotations_file_path = ROOT / session / f'{session}.json'
source_type = 'rgb'

In [ ]:
extract_frames(
    input_video_path=input_video_path,
    annotations_file_path=annotations_file_path,
    force_overwrite=False,
    source_type=source_type,
)